# Dask for NLP

In [1]:
import dask.dataframe as dd
import coiled

## 1. Spin up Cluster

In [ ]:
coiled.create_software_environment(
    conda="environment.yml"
)

Creating new software environment
Creating new ecr build
STEP 1: FROM coiled/default:sha-6b4e896
STEP 2: COPY environment.yml environment.yml
--> 72f35ebbbdf
STEP 3: RUN conda env update -n coiled -f environment.yml     && rm environment.yml     && conda clean --all -y     && echo "conda activate coiled" >> ~/.bashrc
Solving environment: ...working... done

python-3.9.7         | 18.6 MB   | ########## | 100% 
sniffio-1.2.0        | 15 KB     | ########## | 100% 
requests-2.27.1      | 54 KB     | ########## | 100% 
heapdict-1.0.1       | 8 KB      | ########## | 100% 
pysocks-1.7.1        | 31 KB     | ########## | 100% 
importlib-metadata-4 | 39 KB     | ########## | 100% 
libpng-1.6.37        | 278 KB    | ########## | 100% 
pyzmq-22.3.0         | 470 KB    | ########## | 100% 
markupsafe-2.0.1     | 22 KB     | ########## | 100% 
aiosignal-1.2.0      | 12 KB     | ########## | 100% 
fsspec-2021.11.0     | 91 KB     | ########## | 100% 
partd-1.2.0          | 19 KB     | ########## 

In [4]:
cluster = coiled.Cluster(
    name="dask-nlp",
    software="dask-nlp",
    n_workers=50,
    worker_cpu=8,
    worker_memory="24Gib",
    backend_options={'spot':'True'}
)

Output()

Using existing cluster: 'dask-nlp'

In [5]:
from distributed import Client
client = Client(cluster)
client

/Users/rpelgrim/mambaforge/envs/dask-nlp/lib/python3.9/site-packages/distributed/client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+----------------+---------------+---------------+
| Package     | client         | scheduler     | workers       |
+-------------+----------------+---------------+---------------+
| dask        | 2022.02.0      | 2021.11.1     | 2021.11.1     |
| distributed | 2022.02.0      | 2021.11.1     | 2021.11.1     |
| msgpack     | 1.0.3          | 1.0.2         | 1.0.2         |
| numpy       | 1.21.5         | 1.21.4        | 1.21.4        |
| pandas      | 1.4.1          | 1.3.4         | 1.3.4         |
| python      | 3.9.10.final.0 | 3.9.7.final.0 | 3.9.7.final.0 |
+-------------+----------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tls://10.4.11.178:8786' processes=50 threads=400, memory=1.51 TiB>

## 2. Load Clean Twitter Data

In [7]:
df = dd.read_parquet(
    's3://coiled-datasets/arabic-tweets/arabic_twitter_clean.parquet',
    engine='pyarrow',
    chunksize="64MiB"
)

In [8]:
df.head()

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/opt/conda/envs/coiled/lib/python3.9/site-packages/pandas/_libs/internals.cpython-39-x86_64-linux-gnu.so'>